# Isochrones: tests, overlays, isolignes, masques

## Table des matières

* [*** 1. Exploration des données issues des requêtes journeys de Navitia pour essayer de recréer un isochrone***](#explore_data)

    * [1.1. Preparation](#prepa)
    
    * [1.2. Utilisation de la librairie développée spécifiquement](#ut_lib)
    
        * [*1.2.1. Lancement des premiers calculs*](#prem_calcs)
    
        * [*1.2.2.Importer les lignes et les points du réseau pour les cartographier*](#carto_reseau)
    
    * [1.3. Exploration des résultats: cartographie de tous les chemins minimaux](#explo_ch_min)
    
    * [1.4. Exploration des résultats: cartographie des durées et des distances à pied](#explo_carto)

* [***2. Etude de variations de représentations***](#var_repr)

    * [2.1 Différentes manières de représenter les isochrones](#diff_man_iso)

        * [*2.1.1. Les buffers autour des stations et leur union*](#buff_stations)

    * [2.2Isochrones et réseau viaire: isolignes "*bufferisées*" et opérations spatiales (*difference, union, intersection, symmetric_difference*)](#iso_viaire)

        * [*2.2.1. Récupérer un réseau viaire sous la forme d'un graphe*](#recup_reseau)

        * [*2.2.2. Des fonctions pour mesurer des temps de calculs*](#fonc_calc)

        * [*2.2.3. Import des modules et lancement du programme à partir du fichier de paramètres*](#lanc_prog)

        * [*2.2.4. Récupération des résultats et formatage pour les glyphs *multi_polygons* Bokeh*](#format_bokeh)

        * [*2.2.4. Cartographie des résultats*](#bokeh_map)

        * [*2.2.5. Export en svg*](#export_svg)

* [***3. Couches et masques (*en cours*)***](#layer_mask)



<a id="iso_viaire"></a>
### 2.2 Isochrones et réseau viaire: isolignes "*bufferisées*" et opérations spatiales (*difference, union, intersection, symmetric_difference*)

<a id="recup_reseau"></a>
#### 2.2.1 Récupérer un réseau viaire sous la forme d'un graphe 

> Avertissement: si vous souhaitez tester le code ci-dessous, veillez à ne pas mettre une distance trop importante. Pour donner un exemple, la distance entrée initialement est de 50 000 mètres et cela prend entre 40 et 50 minutes pour récupérer le graphe. C'est pour cette raison que nous écrivons le graphe sous la forme de 2 fichiers JSON (*noeuds et arcs*). 

In [ ]:
# Add module to path
import os
import sys
import time
import json
import osmnx as ox

add_path = os.getcwd().replace('experiments', '')
sys.path.append(add_path)

from osmnx_based_functions import graph_with_time


edges_path = os.path.join(add_path, "graphs\Paris_50km_edges.json")
nodes_path = os.path.join(add_path, "graphs\Paris_50km_nodes.json")

point = 2.3467922,48.8621487

distance = 50000
G = graph_with_time(point, distance, edges_path, nodes_path, epsg={"init":"epsg:3857"})


<a id="fonc_calc"></a>
#### 2.2.2 Des fonctions pour mesurer des temps de calculs

In [1]:
import time

def seconds_to_time(seconds, option="all"): 
    milliseconds = int(round(seconds * 1000))
    s, ms = divmod(milliseconds, 1000)
    m, s = divmod(s, 60)
    h, m = divmod(m, 60)
    time_format = "%02d:%02d:%02d:%02d" % (h, m, s, ms)
    
    if option == "all":
        return time_format, milliseconds
    elif option == "ms":
        return milliseconds
    elif option == "format":
        return time_format
    

def time_profile(start, option="all"):
    end = time.time()
    seconds = seconds_to_time(end - start, option=option)
    
    return seconds

<a id="lanc_prog"></a>
#### 2.2.3 Import des modules et lancement du programme à partir du fichier de paramètres

In [2]:
# Add module to path
import os
import sys
import time
import json

add_path = os.getcwd().replace('experiments', '')
sys.path.append(add_path)

from dotenv import load_dotenv
import docopt
import json
from pathlib import Path
from csv_to_json import csv_to_json
from isos_and_intersections import GetIso

param_csv = os.path.join(add_path, "params/test_journeys.csv")
param_json = os.path.join(add_path, "params/test_journeys.json")
places_cache = os.path.join(add_path, "params/places_cache.json")

#Parameters
env_path = Path(add_path) / '.env'
load_dotenv(dotenv_path=env_path)
TOKEN = os.getenv("NAVITIA_TOKEN")

columns_with_array_of_str = [
    "addresses",
    "excluded_modes",
    "durations"
    ]

#JSON INPUT
json_file = csv_to_json(param_csv, param_json, ";", columns_with_array_of_str)

start = time.time()
gdf_global = GetIso(param_json, places_cache, api="navitia", token=TOKEN).get_all_isos()
print ("TOTAL GDF_GLOBAL",time_profile(start, option = "format"))

URL https://api.navitia.io/v1/coverage/fr-idf/journeys?from=2.3447555;48.8511485&datetime=2019-01-17T08:00&max_duration=1200


C:\Users\thomas\Documents\code\iso\app_iso\code\osmnx_based_functions.py:201: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  gdf_lines.drop_duplicates(subset=["source","target"],inplace=True)
C:\Users\thomas\Documents\code\iso\app_iso\code\osmnx_based_functions.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  gdf_lines['xs'] = gdf_lines.apply(getLineCoords, geom='line', coord_type='x', axis=1)
C:\Users\thomas\Documents\code\iso\app_iso\code\osmnx_based_functions.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

polys 00:00:07:267
Overlays 00:01:04:469
TOTAL GDF_GLOBAL 00:01:47:785


<a id="format_bokeh"></a>
#### 2.2.4 Récupération des résultats et formatage pour les glyphs *multi_polygons* Bokeh

In [14]:
from shapely.ops import cascaded_union
from geo_functions import multi_gdf_to_multi_bokeh as mbokeh

id_ = "all_08h00_Chatelet"
address = "90, Boulevard Saint-Germain, 75005, France"
duration = 20
key = id_, address, duration

lines = gdf_global["isolines"][key]["gdf_lines"]
polys_union = gdf_global["isolines"][key]["polys_union"]
sym_diff = gdf_global["isolines"][key]["symmetric_difference"]
difference = gdf_global["isolines"][key]["difference"]
intersection = gdf_global["isolines"][key]["intersection"]
union = gdf_global["isolines"][key]["union"]
isochrones = gdf_global["isolines"][key]["isochrones"]

data = {
    "union":mbokeh(union),
    "difference":mbokeh(difference),
    "lines_buffer":mbokeh(polys_union),
    "sym_diff":mbokeh(sym_diff),
    "intersection": mbokeh(intersection),
    "isochrones": mbokeh(isochrones)
}


In [19]:
polys_union_ = polys_union.to_crs({'init': 'epsg:4326'})
polys_union_[["geometry"]].to_file("buffered_lines.geojson", driver="GeoJSON")

isochrones.crs = {'init': 'epsg:3857'}
isochrones_ = isochrones.to_crs({'init': 'epsg:4326'})
isochrones[["geometry"]].to_file("isochrones.geojson", driver="GeoJSON")

<a id="bokeh_map"></a>
#### 2.2.5 Cartographie des résultats
> * ***Union*** => Union des isochrones et des isolignes bufferisées
> * ***Difference*** => Difference entre les isochrones et les isolignes bufferisées
> * ***Lines_buffer*** => Lignes bufferisées
> * ***Symm_diff*** => Difference symétrique entre les isochrones et les isolignes bufferisées
> * ***Intersection*** => Intersection entre les isochrones et les isolignes bufferisées
> * ***Isochrones*** => Isochrones

> *Pour plus d'informations sur les méthodes d'overlays, voir la [documentation de GeoPandas](http://geopandas.org/set_operations.html)*

In [14]:
add_path = os.getcwd().replace('experiments', '')
sys.path.append(add_path)

import geopandas as gpd
import geojson
from bokeh.plotting import figure, show
from bokeh.palettes import Viridis256, Magma11, RdYlBu11, Greens9, Inferno256, Viridis6
from bokeh.io import output_notebook
from bokeh.tile_providers import STAMEN_TONER_BACKGROUND
from bokeh.models import GeoJSONDataSource, ColumnDataSource, CategoricalColorMapper, LinearColorMapper
from bokeh.palettes import Colorblind8 as cb
from bokeh.transform import factor_cmap
import json
from pyproj import Proj, transform

output_notebook()


#Prepare the Bokeh figure
# color_mapper = LinearColorMapper(palette=Viridis4)
p = figure(
    title="Overlays"
)

p.width = 800
p.add_tile(STAMEN_TONER_BACKGROUND, alpha=0.2)


#Set origin
lat = 2.34472
lng = 48.85103
inProj = Proj(init='epsg:4326')
outProj = Proj(init='epsg:3857')
lat,lng = transform(inProj,outProj,lat,lng)
data_ori = {
    "x":[lat],
    "y":[lng],
    "name":["Origin"]
}

ori = ColumnDataSource(data_ori)

#Prepare overlays
i=0
for name,value in data.items():
    p.multi_polygons(
    xs=value['xs'],
    ys=value['ys'],
    color=Viridis6[i],
    alpha=0.5,
    legend=name
    )
    i+=1

#Add ori
p.triangle(
    x="x",
    y="y",
    size=20,
    color="red",
    alpha=0.5,
    source=ori,
    legend="Origin"
)

p.legend.click_policy="hide"


show(p)

Loading BokehJS ...